# 02 - Plot the Comparisons

Here, we plot the difference between the global average fields from model years 081 to 100

## Imports

In [ ]:
from distributed import Client
from ncar_jobqueue import NCARCluster
import xarray as xr
from cartopy import crs
import holoviews as hv

import holoviews as hv
import geoviews as gv
import datashader
from holoviews.operation.datashader import regrid, shade, datashade, rasterize
import matplotlib.pyplot as plt
import panel as pn

pn.extension()
hv.extension('bokeh')

## Spin Up a Dask Cluster

In [ ]:
cluster = NCARCluster()
cluster.scale(10)
client = Client(cluster)

In [ ]:
client

## Setup Plotting Functions

In [ ]:
def plot_interactive(ds, variable, cmap='magma'):
    """Plots an interactive plot using holoviews"""
    da = ds[variable]
    da['nlon'] = ds.nlon
    da['nlat'] = ds.nlat
    da = da.drop_vars(['TLAT', 'TLONG', 'ULAT', 'ULONG'])
    
    hvds = hv.Dataset(da, kdims=list(da.dims))
        
    quadmesh = hvds.to(hv.QuadMesh, kdims=["nlon", "nlat"])
    datashade_obj = rasterize(quadmesh, precompute=True, cmap=plt.cm.RdBu_r).opts(colorbar=True, width=600, height=400, cmap=cmap, tools=['hover'])
    return datashade_obj.relabel(f'{ds.title}')

In [ ]:
def plot_comparison(ds, case_to_compare, variable, baseline = 'b1850.f19_g17.validation_mct.004'):
    """Plot a comparison between cases"""
    
    ds2 = ds.sel(case=baseline)
    ds2.attrs['title'] = baseline
    baseline_plot = plot_interactive(ds2, variable)
    
    ds1 = ds.sel(case=case_to_compare)
    ds1.attrs['title'] = case_to_compare
    case_compare_plot = plot_interactive(ds1, variable)
    
    diff = ds1[[variable]] - ds2[[variable]]
    diff.attrs['title'] = f'{case_to_compare} - {baseline}'
    diff[variable].attrs['units'] = ds[variable].units
    difference_plot = plot_interactive(diff, variable, cmap='seismic')
    
    diff_percent = (diff/ds1[[variable]].max()) * 100
    diff_percent.attrs['title'] = f'Percent Difference'
    diff_percent[variable].attrs['units'] = 'Percent'
    difference_plot_percent = plot_interactive(diff_percent, variable, cmap='seismic')
    
    plots = hv.Layout([case_compare_plot + baseline_plot + difference_plot + difference_plot_percent]).cols(2)
    panel = pn.Column(pn.pane.Markdown(f"## {ds1[variable].long_name} {ds1[variable].units}", align="center"), plots)
    return panel

## Read in the data

In [ ]:
ds = xr.open_zarr('cached_output/averages_year_081_100.zarr')
ds

## Interactive Plots
**Warning**: These plots will only render at a **single level** on the Jupyterbook website. If you are interestd in looking at different vertical levels, please access this notebook on NCAR's computing resources.

They are stored in various sections separated by:
* Physics
* Biogeo

### Potential Temperature

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'TEMP')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'TEMP')

### Salinity

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SALT')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SALT')

### Mixed Layer Depth (HMXL)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'HMXL')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'HMXL')

### Barotropic Stream Fuction (BSF)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'BSF')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'BSF')

### Ice Fraction (IFRAC)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'IFRAC')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'IFRAC')

### Heat Flux (SHF)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SHF')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SHF')

### Freshwater Heat Flux (SFWF)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SFWF')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SFWF')

### Net Shortwave Flux at the Surface (SHF_QSW)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SFWF')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SHF_QSW')

### Phosphate (PO4)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'PO4')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'PO4')

### Vertical Integral of Diatom C Fixation (photoC_diat_zint)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'photoC_diat_zint')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'photoC_diat_zint')

### Vertical Integral of Diazotroph C Fixation (photoC_diaz_zint)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'photoC_diaz_zint')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'photoC_diaz_zint')

### Vertical Integral of Total C Fixation (photoC_TOT_zint)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'photoC_TOT_zint')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'photoC_TOT_zint')

### Vertical Integral of Small Phyto C Fixation

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'photoC_sp_zint')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'photoC_sp_zint')

### POC Flux at 100 m (POC_FLUX_100m)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'POC_FLUX_100m')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'POC_FLUX_100m')

### Diazotroph N Fixation (diaz_Nfix)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'diaz_Nfix')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'diaz_Nfix')

### CaCO3 Flux at 100 m (CaCO3_FLUX_100m)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'CaCO3_FLUX_100m')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'CaCO3_FLUX_100m')

### SiO2 Production (SiO2_PROD)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SiO2_PROD')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SiO2_PROD')

### Alkalinity (ALK)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'ALK')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'ALK')

### Dissolved Inorganic Carbon (DIC)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'DIC')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'DIC')

### Dissolved Oxygen (O2)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'O2')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'O2')

### Dissolved Organic Carbon (DOC)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'DOC')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'DOC')

### Refractory DOC (DOCr)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'DOCr')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'DOCr')

### DIC Surface Gas Flux (FG_CO2)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'FG_CO2')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'FG_CO2')

### Dissolved Ammonia (NH4)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'NH4')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'NH4')

### Dissolved Inorganic Nitrate (NO3)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'NO3')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'NO3')

### Dissolved Inorganic Silicate (SiO3)

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_mct.002', 'SiO3')

In [ ]:
plot_comparison(ds, 'b1850.f19_g17.validation_nuopc.004', 'SiO3')